## 1. Scrape of all matches of 2021 Brazilian soccer championship

In [244]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd
import re

def uol_matches(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req).read()
    page = soup(html, 'html.parser')
    containers = page.findAll('ul', class_ = 'rounds-content')
    tag = containers[0].text

    if re.search('encerrado           A definir', tag) != None:
        part1, part2 = re.split('encerrado           A definir', tag)[:2]
        part2_split = re.split('A definir', part2)
        part3 = []
        for part in part2_split:
            part = part.split('  ')
            part = part[2] + ' - * - * - ' + part[1]
            part3.append(part)
        part2 = "            ".join(part3)
        tag = part1 + part2
        tag

    games = re.split('Sáb|Dom|Seg|Ter|Qua|Qui|Sex|pós-jogo', tag)
    games = [g.replace(',', ' ').strip() for g in games if g.strip() != '']

    for i in range(len(games)):
        res1 = re.search(r'^\d\d/\d\d\s-\s\d\dh\d*', games[i])
        if i<len(games)-1: res2 = re.search(r'^\d\d/\d\d\s-\s\d\dh\d*', games[i+1])
        if res1 == None and res2 != None: 
            games[i] = res2[0] + ' - ' + games[i]
    games = [re.sub(r'\s{3,9}', ' - ', g).strip() for g in games if len(g.split('-')) > 2]
    games = [re.split(r' - |\(\)', g) for g in games]
    df = pd.DataFrame(games).iloc[:,:5]
    df.columns = ['Date', 'Time', 'Place', 'Team1', 'Team2']
    df[['Score1', 'Score2']] = ''

    for i in range(len(df)):
        txt1 = df.loc[i,'Team1'].strip()[3:]
        score1 = re.findall(r'\d+|-$', txt1)[0]
        if score1== '-': score1=''
        txt2 = df.loc[i,'Team2'].strip()[:-3]
        score2 = re.findall(r'^\d+|^-', txt2)[0]
        if score2== '-': score2=''
        df.loc[i, 'Team1'] = re.split(r'\d+|-$', txt1)[0]
        df.loc[i, 'Team2'] = re.split(r'^\d+|^-', txt2)[1]
        df.loc[i, 'Score1'] = score1
        df.loc[i, 'Score2'] = score2
    df.replace('*', '', inplace=True)
    return df

### 1.1 Show all 380 match results of A-league

In [247]:
df = uol_matches('https://www.uol.com.br/esporte/futebol/campeonatos/brasileirao')
df.to_excel('matches_brasileirao.xlsx', index=False)
df

,Date,Time,Place,Team1,Team2,Score1,Score2
0,29/05,19h,Arena Pantanal,Cuiabá,Juventude,2,2
1,29/05,20h,Pituaçu,Bahia,Santos,3,0
2,29/05,21h,Morumbi,São Paulo,Fluminense,0,0
3,30/05,11h,Mineirão,Atlético-MG,Fortaleza,1,2
4,30/05,16h,Maracanã,Flamengo,Palmeiras,1,0
...,...,...,...,...,...,...,...
375,09/12,21h30,Castelão (CE),Fortaleza,Bahia,,
376,09/12,21h30,Arena Pernambuco,Sport,Athletico,,
377,09/12,21h30,Nabi Abi Chedid,Red Bull Bragantino,Internacional,,
378,09/12,21h30,Alfredo Jaconi,Juventude,Corinthians,,


### 1.2 Show all 379 match results of B-league

In [249]:
uol_matches('https://www.uol.com.br/esporte/futebol/campeonatos/serie-b')

,Date,Time,Place,Team1,Team2,Score1,Score2
0,28/05,16h,Bento Freitas,Brasil de Pelotas,Londrina,0,0
1,28/05,19h,Brinco de Ouro,Guarani,Vitória,1,1
2,28/05,21h30,OBA,Vila Nova,Botafogo,1,1
3,28/05,21h30,Aflitos,Náutico,CSA,1,0
4,29/05,11h,São Januário,Vasco,Operário-PR,0,2
...,...,...,...,...,...,...,...
374,28/11,16h,A definir,Cruzeiro,Náutico,,
375,28/11,16h,Moisés Lucarelli,Ponte Preta,Coritiba,,
376,28/11,16h,Hailé Pinheiro,Goiás,Brusque,,
377,28/11,16h,Nilton Santos,Botafogo,Guarani,,


### 1.3 Show all 90 match results of World Cup qualifier in South America

In [250]:
uol_matches('https://www.uol.com.br/esporte/futebol/campeonatos/eliminatorias-sul-americanas')

,Date,Time,Place,Team1,Team2,Score1,Score2
0,08/10,19h30,Defensores del Chaco,Paraguai,Peru,2,2
1,08/10,19h45,Centenário (URU),Uruguai,Chile,2,1
2,08/10,21h30,La Bombonera,Argentina,Equador,1,0
3,09/10,20h30,Metropolitano Roberto Meléndez,Colômbia,Venezuela,3,0
4,09/10,21h30,Neo Química Arena,Brasil,Bolívia,5,0
...,...,...,...,...,...,...,...
85,29/03,,,Peru,Paraguai,,
86,29/03,,,Venezuela,Colômbia,,
87,29/03,,,Bolívia,Brasil,,
88,29/03,,,Chile,Uruguai,,


### 2.1 Show classification of A-league

In [251]:
pd.set_option("display.max_rows", 20, "display.max_columns", 20)

def uol_ranking(url):
    page = pd.read_html(url)
    df = pd.concat([pd.DataFrame(page[0]), pd.DataFrame(page[1])], axis=1)
    df.rename(columns = {'classificação':'Ranking'}, inplace = True)
    df[['Ranking', 'Team']] = df['Ranking'].str.split('°', 1, expand=True)
    df['Team'] = df['Team'].apply(lambda x:re.split(r'[A-Z]{3}$', x)[0].strip())
    cols = df.columns.tolist()
    cols = [cols[0], cols[-1]] + cols[1:-1]
    df = df[cols]
    return df

df = uol_ranking('https://www.uol.com.br/esporte/futebol/campeonatos/brasileirao')
df.to_excel('ranking_brasileirao.xlsx', index=False)
df

,Ranking,Team,PG,J,V,E,D,GC,GP,SG,%
0,1,Atlético-MG,71,32,22,5,5,22,51,29,74
1,2,Flamengo,63,32,19,6,7,28,62,34,66
2,3,Palmeiras,58,33,18,4,11,39,52,13,59
3,4,Red Bull Bragantino,52,34,13,13,8,40,50,10,51
4,5,Corinthians,50,33,13,11,9,32,35,3,51
5,6,Fortaleza,49,33,14,7,12,41,40,-1,49
6,7,Internacional,47,33,12,11,10,35,41,6,47
7,8,Fluminense,45,33,12,9,12,34,31,-3,45
8,9,América-MG,45,33,11,12,10,34,35,1,45
9,10,Ceará,45,33,10,15,8,33,35,2,45


### 2.2 Show classification of B-league

In [252]:
uol_ranking('https://www.uol.com.br/esporte/futebol/campeonatos/serie-b')

,Ranking,Team,PG,J,V,E,D,GC,GP,SG,%
0,1,Botafogo,66,36,19,9,8,29,53,24,61
1,2,Coritiba,64,36,18,10,8,31,47,16,59
2,3,Goiás,61,36,16,13,7,29,44,15,56
3,4,Guarani,59,36,16,11,9,37,52,15,55
4,5,Avaí,58,36,16,10,10,33,40,7,54
5,6,CRB,57,36,15,12,9,36,43,7,53
6,7,CSA,56,36,16,8,12,33,43,10,52
7,8,Náutico,52,36,14,10,12,48,49,1,48
8,9,Vasco,49,37,13,10,14,49,43,-6,44
9,10,Vila Nova,48,37,11,15,11,36,34,-2,43


### 2.3 Show classification of World Cup qualifier in South America

In [253]:
uol_ranking('https://www.uol.com.br/esporte/futebol/campeonatos/eliminatorias-sul-americanas')

,Ranking,Team,PG,J,V,E,D,GC,GP,SG,%
0,1,Brasil,35,13,11,2,0,4,27,23,90
1,2,Argentina,29,13,8,5,0,6,20,14,74
2,3,Equador,23,14,7,2,5,13,23,10,55
3,4,Colômbia,17,14,3,8,3,17,16,-1,40
4,5,Peru,17,14,5,2,7,20,15,-5,40
5,6,Chile,16,14,4,4,6,16,15,-1,38
6,7,Uruguai,16,14,4,4,6,21,14,-7,38
7,8,Bolívia,15,14,4,3,7,28,20,-8,36
8,9,Paraguai,13,14,2,7,5,18,9,-9,31
9,10,Venezuela,7,14,2,1,11,25,9,-16,17
